In [8]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
import nest_asyncio
import asyncio

nest_asyncio.apply()

# Initialize SerpAPI tool (async)
serpapi = SerpAPIWrapper()

async def search_text(query: str) -> str:
    results = await serpapi.arun(query)
    try:
        snippet = results["organic_results"][0].get("snippet")
        return snippet or results["organic_results"][0].get("title")
    except Exception:
        return "No se encontró texto relevante."

# Configurar LLM OpenAI clásico (usa OPENAI_API_KEY en el entorno)
llm = OpenAI(model="gpt-4o-mini", temperature=0.7)

# Plantilla prompt para generar texto con CoT basado en búsqueda
generate_template = (
    "Eres un escritor creativo que razona paso a paso.\n"
    "Basándote en este texto: '''{search_result}''', escribe un texto de 5 líneas "
    "sobre '{topic}' en estilo {style}."
    )
generate_prompt = PromptTemplate(input_variables=["search_result", "topic", "style"], template=generate_template)
generate_chain = LLMChain(llm=llm, prompt=generate_prompt, output_key="generated_text")

# Plantilla prompt para refinar el texto generado
refine_template = (
    "Ahora eres un editor que pule textos.\n"
    "Pulir el siguiente texto para mejorar claridad y estilo:\n{generated_text}"
    ")
refine_prompt = PromptTemplate(input_variables=["generated_text"], template=refine_template)
refine_chain = LLMChain(llm=llm, prompt=refine_prompt, output_key="refined_text")

# Ejecutar las chains secuencialmente dentro del loop async usando threads para no bloquear el event loop
async def main():
    query = "la luna"
    search_result = await search_text(query)
    # Ejecutar generate_chain de forma sincrónica en un thread
    generated_text = await asyncio.to_thread(generate_chain.run, {"search_result": search_result, "topic": "la luna", "style": "poético y evocativo"})
    # Ejecutar refine_chain con el resultado anterior
    refined_text = await asyncio.to_thread(refine_chain.run, {"generated_text": generated_text})
    print("Texto generado:\n", generated_text)
    print("\nTexto refinado:\n", refined_text)

await main()

SyntaxError: unterminated string literal (detected at line 37) (4085783293.py, line 37)